In [2]:
import pymysql
import configparser
import pandas as pd

In [3]:
parser = configparser.ConfigParser()
parser.read('pipeline.conf')

# Credentials for MySQL

hostname = parser.get("mysql_config", "hostname")
port = parser.get("mysql_config", "port")
username = parser.get("mysql_config", "username")
password = parser.get("mysql_config", "password")

In [4]:
conn = pymysql.connect(
    host=hostname,
    user=username,
    password=password,
    port=int(port)
)
if conn is None:
    print("Connection failed")
else:
    print("Connection established!")

Connection established!


In [5]:
cursor = conn.cursor()

In [6]:
cursor.execute("SHOW DATABASES")
cursor.fetchall()

(('board_games',),
 ('datapipe',),
 ('desafio',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sakila',),
 ('sys',))

In [7]:
cursor.execute("USE board_games")

0

In [8]:
cursor.execute("SHOW TABLES")
cursor.fetchall()

(('players_data',),)

In [9]:
cursor.execute("DESCRIBE players_data")
cursor.fetchall()

(('id', 'int', 'NO', 'PRI', None, 'auto_increment'),
 ('jogador_id', 'int', 'YES', '', None, ''),
 ('genero', 'varchar(20)', 'YES', '', None, ''),
 ('data_nascimento', 'date', 'YES', '', None, ''),
 ('jogo', 'varchar(50)', 'YES', '', None, ''),
 ('pais', 'varchar(50)', 'YES', '', None, ''),
 ('num_vitorias', 'int', 'YES', '', None, ''),
 ('num_derrotas', 'int', 'YES', '', None, ''),
 ('total_partidas', 'int', 'YES', '', None, ''))

In [24]:
# pais com menor media de vitorias

cursor.execute("""
    SELECT pais, AVG(num_vitorias) as nv
    FROM players_data
    GROUP BY pais
    ORDER BY nv ASC
""")
               
cursor.fetchone()

# Paraguai


('Paraguai', Decimal('21.9754'))

In [37]:
# soma derrotas brasil

cursor.execute("""
    SELECT pais, SUM(num_derrotas) as soma_vit_bra
    FROM players_data
    WHERE pais = 'Brasil'
""")

cursor.fetchone()

# 2038

('Brasil', Decimal('2038'))

Qual é o segundo jogo preferido entre as mulheres?

In [68]:
cursor.execute("""

    WITH count AS
    (
    SELECT jogo, count(jogo) as cnt
    FROM players_data
    WHERE genero = 'Feminino'
    GROUP BY jogo
    ),
        ranked AS (
        SELECT *, RANK() OVER(ORDER BY cnt DESC) as ranking
        FROM count
        )
SELECT * FROM ranked
WHERE ranking = 2
""")

cursor.fetchall()

# Xadrez with 87 games

(('Xadrez', 87, 2),)

Qual o jogo que possui o maior número de partidas?

In [71]:
cursor.execute("""
    SELECT jogo, count(total_partidas) as num_partidas
    FROM players_data
    GROUP BY jogo
    ORDER BY num_partidas DESC
""")

cursor.fetchall()

# Xadrez with 171

(('Xadrez', 171), ('Dama', 164), ('Jogo da velha', 164))